# Quick Overview <a class="anchor" id="top"></a>

- **Prepared by:**

    - [Yalin Li](https://qsdsan.readthedocs.io/en/latest/authors/Yalin_Li.html)

- **Covered topics:**

    - [1. Introduction and Installation](#s1)
    - [2. Download External Databases](#s2)
    - [3. Get Characterization Factors](#s3)

In [1]:
# To change directory to the cloned bw2qsd (as opposed to the pip installed version)
# import os
# path = os.path.realpath('')+'/../'
# os.chdir(path)

In [2]:
import bw2qsd as bq
print(f'This tutorial was made with bw2qsd v{bq.__version__} '
      f'and brightway2 v{bq.bw2_version}.')

ImportError: cannot import name 'eidl' from partially initialized module 'bw2qsd' (most likely due to a circular import) (/Users/yalinli_cabbi/opt/anaconda3/envs/bw2/lib/python3.8/site-packages/bw2qsd/__init__.py)

## 1. Introduction and Installation <a class="anchor" id="s1"></a>

### 1.1. Introduction

`bw2qsd` is (currently) a lite package leveraging the [``Brightway2`` platform](https://2.docs.brightway.dev/index.html) for convenient importing of life cycle impact characterization factors (CFs) and use them in packages developed for quantitative sustainable design (QSD) of technologies and systems hosted on [the GitHub homepage of QSD-Group](https://github.com/QSD-Group). You can certainly use this package to just extract the CF values and use in other ways you like.

Note that `Brightway2` is a powerful platform with many capacities and `bw2qsd` is not intended to leverage all of them, as the name suggests, `bw2qsd` is tailored for the QSD. 

There are still some manual work required to clean up the data and format them into the requirement of QSD pcakges, but the entire process can be automated with future development of both `bw2qsd` and QSD packages.

Additionally, you will find this tutorial much easier to follow if you have familiarized yourself with the basic concepts in `Brightway2`.

### 1.2. Installation

The easiest way to install `bw2qsd` is through pip:

```bash
pip install bw2qsd
```

You can also clone this repo and install the dependent packages in setup.py, but note that the release of `brightway2` is not very stable (i.e., some versions might not work).

### 1.3. Other important notes

1. When running ``BW2QSD``, you might run into a `Pickle deserialization error` related to an outdated pickle file.

To prevent from getting this error, you can run the following code before you quitting the program (i.e., AFTER you have finished getting CFs), like suggested at the end of this tutorial.

```bash
from bw2qsd import remove_setups_pickle
remove_setups_pickle()
```

If you forgot to remove the pickle file, `bw2qsd` will help you to fix it (by removing the outdated pickle file) the next time you run it.

2. If there are functions in ``Brightway2`` packages that you would like to use but not included in ``BW2QSD`` (e.g., renaming databases), you can import the corresponding packages and for those tasks.

## 2. Download External Databases <a class="anchor" id="s2"></a>

---
**Note**
Currently only the [ecoinvent](http://ecoinvent.org) (license and login credentials required) and [forwast](https://lca-net.com/projects/show/forwast/) databases have been developed and tested.

In the future, databases in the [Federal LCA Commons repository](https://www.lcacommons.gov/lca-collaboration/) will be included.

---

In [ ]:
from bw2qsd import DataDownloader

In [ ]:
# Only the most basic steps are shown in this tutorial,
# if you want look at the full document, you can use
help(DataDownloader)

In [ ]:
# or (run this line)
DataDownloader?

In [ ]:
# Firstly initiate the class
downloader = DataDownloader()

You will be prompted for your ecoinvent credentials in this process, you will also be able to select the version of the ecoinvent database and the system model (refer to the document of this function for explanation on system models).

Downloading the database and extracting the data can take tens of minutes, you will get print-out messages along the way.

When the process is finished, you will get a message with the name of the database, it will be in the format of "ecoinvent_XY" with X being system model and Y being the version, e.g., "ecoinvent_cutoff371" means the cut-off system model and version 3.7.1.

In [ ]:
# Uncomment to run the downloader
# downloader.download_ecoinvent()

In [ ]:
# You can check what databases you have by
downloader.available_databases

## 3. Get Characterization Factors <a class="anchor" id="s3"></a>

### 3.1. Load database

In [ ]:
from bw2qsd import CFgetter

In [ ]:
# Let's assume we have the "ecoinvent_apos371" database
ei = CFgetter('ei')
ei.load_database('ecoinvent_apos371')

### 3.2. Load impact indicator

In [ ]:
# To have a look at all available impact assessment methods and associated indicators (800+),
# you can use (`add` is whether to add the returned indicators to calculation)
ei.load_indicators(add=False)

In [ ]:
# You can use the `method`, `category`, and `indicator` keywords to narrow down the ones you like
# assumess we want all TRACI indicators
ei.load_indicators(add=True, method='TRACI', category='', indicator='')

In [ ]:
# It is not clear why most of the TRACI indicators are marked as "obsolete"
ei.indicators

In [ ]:
# If you want to remove one or more indicator(s)
ei.remove('indicators', (('TRACI (obsolete)', 'human health', 'non-carcinogenics'),))

### 3.3. Load activities

In [ ]:
# Next is to look for the activities of interest,
# assume we want to look at some building-related impacts
act_dct = ei.load_activities(string='building', add=False, limit=10)
act_dct

In [ ]:
# If we want to look at the detailed descriptions of the activity
ei.show_activity(act_dct['building construction, hostel'])

In [ ]:
# Let's just assume that we want to load all of returned building-related activities
ei.load_activities('building', True)

In [ ]:
info = ei.show_activity('building construction, hostel')
info

In [ ]:
# To look at the full description
info['building construction, hostel']['comment']

### 3.4. Get charachterization factors

In [ ]:
# Finally, we can get the CF values (the calculation may take minutes)
# if you provide a path, the results will be saved as a tsv (`sep='\t'`)/csv or an Excel
# (depending on the file type you provided in the path),
# a sample output (sample_output.tsv) is provided in the package.
df = ei.get_CFs(show=False, path='')
df

In [ ]:
# Before quitting the program, removing the setups.pickle file will prevent you from getting an error
# (related to the ``bw2data`` package) the next time you load BW2QSD
from bw2qsd import remove_setups_pickle
remove_setups_pickle()